En este documento vamos a hacer los cambios requeridos en xml: 

    Cambiar el numero de gender por el nombre correspondiente
    
    Eliminar el "Level 0"

In [3]:
import re
import os
import xml.etree.ElementTree as ET

tree = ET.parse('data_xml.xml')
root = tree.getroot()


xml_list = []
for child in root:
    xml_dict = {}
    for subchild in child:
        xml_dict[subchild.tag] = subchild.text
    xml_list.append(xml_dict) # Es una lista que contiene diccionarios

In [4]:
#Hacemos la primera prueba, cogiendo solo un elemento del fichero 
xml_list [0]['gender'] = "Man"

In [5]:
type(xml_list)

list

In [6]:
#Nuestro fichero es una lista de diccionarios. 
#Hemos realizado una función iterando por los elementos de nuestra lista (que son diccionarios) y cambiando el value de la key "gender"
#Hemos identificado una columna redundante, que era "Level0" y hemos utilizado un bucle for tambien para que lo elimine en todos los diccionarios.

def limpieza_xml (lista):
    for diccionario in lista:

        if diccionario['gender'] == "0" :
            diccionario["gender"] = "Man"
        elif diccionario["gender"] == "1":
            diccionario["gender"] = "Woman"
        elif diccionario['gender'] == "2":
            diccionario['gender'] = "Nonbinary"
        elif diccionario['gender'] == "3":
            diccionario['gender'] = "Prefer not to say"
        elif diccionario['gender'] == "4":
            diccionario['gender'] = "Prefer to self-describe"
    
    for diccionario in lista:
        diccionario.pop("level_0")

    return lista
    


In [7]:
limpieza_xml(xml_list)

[{'index': '1', 'time': '784', 'age': '50-54', 'gender': 'Man'},
 {'index': '2', 'time': '924', 'age': '22-24', 'gender': 'Man'},
 {'index': '3', 'time': '575', 'age': '45-49', 'gender': 'Man'},
 {'index': '4', 'time': '781', 'age': '45-49', 'gender': 'Man'},
 {'index': '5', 'time': '1020', 'age': '25-29', 'gender': 'Woman'},
 {'index': '6', 'time': '141', 'age': '18-21', 'gender': 'Woman'},
 {'index': '7', 'time': '484', 'age': '30-34', 'gender': 'Man'},
 {'index': '8', 'time': '1744', 'age': '22-24', 'gender': 'Man'},
 {'index': '9', 'time': '655', 'age': '30-34', 'gender': 'Man'},
 {'index': '10', 'time': '1777', 'age': '40-44', 'gender': 'Man'},
 {'index': '11', 'time': '3081', 'age': '18-21', 'gender': 'Woman'},
 {'index': '12', 'time': '1922', 'age': '18-21', 'gender': 'Woman'},
 {'index': '13', 'time': '852', 'age': '45-49', 'gender': 'Man'},
 {'index': '14', 'time': '838', 'age': '22-24', 'gender': 'Man'},
 {'index': '15', 'time': '563', 'age': '35-39', 'gender': 'Man'},
 {'ind

In [22]:


def sacar_values(xml_list):
    
    values = []

    for i in xml_list:
        values.append(i.values())
        
    return values





In [23]:
values_xml = sacar_values(xml_list)

In [25]:
values_xml

[dict_values(['1', '784', '50-54', 'Man']),
 dict_values(['2', '924', '22-24', 'Man']),
 dict_values(['3', '575', '45-49', 'Man']),
 dict_values(['4', '781', '45-49', 'Man']),
 dict_values(['5', '1020', '25-29', 'Woman']),
 dict_values(['6', '141', '18-21', 'Woman']),
 dict_values(['7', '484', '30-34', 'Man']),
 dict_values(['8', '1744', '22-24', 'Man']),
 dict_values(['9', '655', '30-34', 'Man']),
 dict_values(['10', '1777', '40-44', 'Man']),
 dict_values(['11', '3081', '18-21', 'Woman']),
 dict_values(['12', '1922', '18-21', 'Woman']),
 dict_values(['13', '852', '45-49', 'Man']),
 dict_values(['14', '838', '22-24', 'Man']),
 dict_values(['15', '563', '35-39', 'Man']),
 dict_values(['16', '1315', '50-54', 'Man']),
 dict_values(['17', '479', '18-21', 'Man']),
 dict_values(['18', '249', '22-24', 'Man']),
 dict_values(['19', '650', '30-34', 'Man']),
 dict_values(['20', '1461', '70+', 'Man']),
 dict_values(['21', '551', '25-29', 'Woman']),
 dict_values(['22', '258', '30-34', 'Man']),
 dic

In [29]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto1')
mycursor = cnx.cursor()

sql = "INSERT INTO tabla_xml (index_xml, time, age, gender) VALUES (%s, %s, %s, %s)"

val = (('1',) ,('784',) ,('50-54',) ,('Man',))
try:
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

Not enough parameters for the SQL statement
Error Code: -1
SQLSTATE None
Message Not enough parameters for the SQL statement
